In [1]:
import json
import tkinter as tk
from tkinter import *
from PIL import ImageTk, Image
from pyspark import SparkContext, SparkConf
from xlrd import open_workbook
from fuzzywuzzy import fuzz
import pandas as pd
import json
import re

In [2]:
conf = SparkConf().setAppName("DecentWorkandEconomic")
sc = SparkContext(conf=conf)

In [3]:
path = "file:///Users/ritukaushik/Downloads/GOTTweets/*.csv"
textRDD = sc.textFile(path)
tweetRDD = textRDD.map(lambda x:x)
rdd1 = tweetRDD.map(lambda x: (x,x.split(";")))
rdd2 = rdd1.map(lambda x: (x[1][0],x[0]))

In [4]:
places_path="file:///Users/ritukaushik/Downloads/TouristPlaces.xlsx"
dfs = pd.read_excel(places_path, sheetname=None)
places=list(dfs.items())
rdd_places=sc.parallelize(places)
tourist_spots=rdd_places.map(lambda x: (x[0],(x[1].values)))
ts=tourist_spots.flatMap(lambda x:[(x[1][i], x[0]) for i in range(len(x[1]))])
ts_formatted=ts.map(lambda x: (x[0][0], (x[0][1],x[0][2],x[1])))
ts_info=ts_formatted.filter(lambda x: str(x[0]).strip() not in ('nan', 'Name'))

In [5]:
# ts_info.collect()

In [6]:
i = 0
def columnExtract(fileName):
    book = open_workbook(fileName)
    column1 = list()
    for xl_sheet in book.sheets():
        raw_data = list()
        for row in range(1, xl_sheet.nrows):
            raw_data.append(xl_sheet.row(row))
        places_list = list()
        for raw in raw_data:
            places_list.append(list())
            for r in raw:
                places_list[-1].append(r.value)
        for i in places_list:
            if len(i[0])>0:
                column1.append(i[0])  
    column1 = [ ''.join(x.split()).lower() for x in column1 ]    
    return column1
columns = columnExtract('TouristPlaces.xlsx')
columns = [ ''.join(x.split()).lower() for x in columns]

In [7]:
def extract(line,columns):
    line = re.sub("[^a-zA-Z0-9]",' ',line.lower())
    lines = line.split()
    return list(set(lines) & set(columns))

In [8]:
def gettweetcount(row):   
    count =[0 for i in range(12)]  
    for i in range(len(row)):
        count[int(row[i][0])-1] = row[i][1]
    return count

In [9]:
rdd3 = rdd2.map(lambda x: [x[0], extract(x[1],columns)]).filter(lambda x: len(x[1]) > 0)

In [10]:
rdd4 = rdd3.map(lambda x : ( (x[1][0],  x[0].split('-')[1]),1 ))

In [11]:
rddpalcescount =  rdd4.reduceByKey(lambda x,y:x+y).map(lambda x: (x[0][0],[(x[0][1],x[1])])) 

In [12]:
rddpalcesmonthcount = rddpalcescount.reduceByKey(lambda x,y: x+y) 

In [13]:
Placescount =  rddpalcesmonthcount.map(lambda x: (x[0],gettweetcount(x[1])))

In [14]:
Placescount = Placescount.map(lambda x:(x[0],x[1]))  

In [15]:
ts_info = ts_info.map(lambda x: ( ''.join(x[0].split()).lower(),x[1] ) )

In [16]:
ts_info.take(3)

[('onedirection', ('Museums', 'Indianapolis', 'Indiana')),
 ('fortwanechildrenzoo', ('Zoos', 'Fort Wayne', 'Indiana')),
 ('basilicaofthedacredheart', ('Church', 'South Bend', 'Indiana'))]

In [17]:
merged=Placescount.union(ts_info).groupByKey().mapValues(list)

In [18]:
merged = merged.filter(lambda x: len(x[1])==2)

In [19]:
# merged.take(100)

In [20]:
ts_infoc =  ts_info.filter(lambda x: len(x[1]) ==3 )
ts_infoc.count()

1537

In [21]:
def checkcount(row):
    if(len(row)==2):
        return row
    res = []
    count =[0 for i in range(12)]
    res.append(row[0])
    res.append(count)
    return res

In [22]:
import numpy as np
import hashlib
def lshband(rddrow,bucket):
    feature  = rddrow[1]
    bands = min(len(feature),10)
    bands = 3
    result = []
    chuncks = np.array_split(feature,bands)    
    for i in range(0,bands):
        hashval = hashlib.md5(chuncks[i])
        result.append((rddrow[0],int('0x'+hashval.hexdigest(),16)%bucket + i*bucket))
    return result

In [23]:
def addCounts(x,y):
    return x

In [24]:
ts_months_vector = merged.map(lambda x: (x[0],checkcount(x[1]))).map(lambda x:(x[0],( x[1][0],x[1][1])))

In [25]:
################ Recommendation Systems

In [26]:
# Places Similarity

In [27]:
ts_months_vectorplaces = ts_months_vector.map(lambda x:(x[0],x[1][0]))  

In [28]:
ts_months_vectorplaceslsh = ts_months_vectorplaces.flatMap(lambda x: lshband(x,1500)) 

In [29]:
ts_months_similarplacesbuckets = ts_months_vectorplaceslsh.map(lambda x:(x[1],[x[0]])).reduceByKey(lambda x,y:list(set(x+y)))

In [30]:
ts_months_similarplaces1 = ts_months_similarplacesbuckets.flatMap(lambda x:[(place,x[1]) for place in x[1]]).reduceByKey(lambda x,y: list(set(x+y))) 

In [31]:
ts_months_similarplaces1.take(3)

[('millcitymuseum',
  ['independencepass',
   'tristancristmagictheatre',
   'milwaukeecountyzoo',
   'arlingtonnationalcemetery',
   'lotzhousemuseum',
   'bellrock',
   'griffithobsevatory',
   'readingterminalmarket',
   'americanvisionaryartmuseum',
   'ritterpark',
   'immaculateconceptionchurch',
   'grandcanyonnorthrim',
   'thebarnesfoundation',
   'archrock',
   'philadelphiamuseumofart',
   'mountpleasantmemorialwaterfrontpark',
   'coronaarch',
   'chainoflakes',
   'easternstatepenitentiary',
   'hooverdam',
   'peninsulastatepark',
   'americanshakespearecenter',
   'grandviewstatepark',
   'longwood',
   'smithrockstatepark',
   'colignybeach',
   'adventuresunlimited',
   'rockcity',
   'tomboftheunknowns',
   'grandcanyonrailway',
   'independencenationalhistoricalpark',
   'smokymountainalpinecoaster',
   'fordycebathhouse',
   'jamestownsettlement',
   'fishcreekfalls',
   'beehivetrail',
   'tahquamenonfallsstatepark',
   'fortadamsstatepark',
   'gettysburgnationalm

In [32]:
featureplaces = ts_months_vectorplaces.filter(lambda x:len(x[1])==12).collectAsMap()

In [33]:
places = ts_months_similarplaces1.collect()

In [34]:
featureplaces

{'360chicagoobservationdeck': [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
 'acadianationalpark': [0, 0, 125, 188, 0, 0, 0, 7, 454, 490, 0, 0],
 'adamsfarm': [0, 0, 7, 3, 0, 0, 0, 0, 11, 4, 0, 0],
 'adventuresunlimited': [0, 0, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0],
 'aerialliftbridge': [0, 0, 0, 1, 0, 0, 0, 0, 9, 0, 0, 0],
 'airzoo': [0, 0, 2, 6, 0, 0, 0, 0, 12, 14, 0, 0],
 'alabamagulfcoastzoo': [0, 0, 0, 10, 9, 1, 0, 0, 1, 0, 3, 1],
 'alcatrazisland': [0, 0, 248, 246, 0, 0, 0, 2, 195, 211, 0, 0],
 'allagashbrewery': [0, 0, 1, 3, 0, 0, 0, 0, 4, 0, 0, 0],
 'alligatoralley': [36, 13, 73, 79, 106, 58, 46, 32, 103, 33, 37, 29],
 'americanmusictheatre': [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
 'americanshakespearecenter': [0, 0, 0, 1, 0, 0, 0, 0, 4, 1, 0, 0],
 'americanswedishinstitute': [0, 0, 0, 3, 0, 0, 0, 0, 1, 3, 0, 0],
 'americanvisionaryartmuseum': [0, 0, 4, 3, 0, 0, 0, 0, 3, 6, 0, 0],
 'andersonjapanesegardens': [0, 0, 5, 20, 20, 5, 0, 0, 7, 13, 0, 0],
 'angeloaktree': [0, 0, 5, 6, 0, 0, 0, 0, 7, 

In [35]:
def  findSimilarplaces(featureplaces,places):
    result = {}
    featdict = featureplaces
    for place in places:
        temp = []
        for similarplace in place[1]:
            if ((place[0] in featdict.keys()) and  similarplace in featdict.keys()):
                temp.append((similarplace,findeuclidean(featdict[place[0]],featdict[similarplace])))      
        temp = sorted(temp, key=lambda x:x[1])# sorting based on distance
        result[place[0]] = temp[0:min(10,len(temp))]
    return result
def findeuclidean(row1,row2):
    cost = 0
    return np.sqrt(np.sum((np.array(row1)-np.array(row2))**2))   

In [36]:
similarplacescost = findSimilarplaces(featureplaces,places) 

In [37]:
similarplacescost

{'360chicagoobservationdeck': [('360chicagoobservationdeck', 0.0),
  ('michiganstadium', 1.0),
  ('snakerivergreenbelt', 1.4142135623730951),
  ('georgiaaquarium', 1.4142135623730951),
  ('cedarpointamusementpark', 1.4142135623730951),
  ('pensacolabeach', 1.4142135623730951),
  ('tristancristmagictheatre', 1.7320508075688772),
  ('lotzhousemuseum', 1.7320508075688772),
  ('atlantabotanicalgarden', 1.7320508075688772),
  ('mountwashingtonautoroad', 1.7320508075688772)],
 'acadianationalpark': [('acadianationalpark', 0.0),
  ('sciencemuseumofminnesota', 696.28514273966812),
  ('waterfrontpark', 801.50670614786497)],
 'adamsfarm': [('adamsfarm', 0.0),
  ('litchfieldbeach', 3.4641016151377544),
  ('musicalinstrumentmuseum', 5.0990195135927845),
  ('chainoflakes', 5.3851648071345037),
  ('presqueislestatepark', 5.5677643628300215),
  ('palmspringsaerialtramway', 5.7445626465380286),
  ('needleshighway', 6.2449979983983983),
  ('littleroundtop', 6.4031242374328485),
  ('carnegiemuseumofnatu

In [38]:
famousspots = tourist_spots.collectAsMap()

In [39]:
# similarplacescost['pensacolabeach']

In [40]:
def findsimilarspots(place):
    res = []
    global similarplacescost
    if place not in similarplacescost.keys():
        return ["places not available"]
    places = similarplacescost[place]
    for similarspot in places:
        res.append(similarspot[0])
    return res[1:min(10,len(res))]

In [41]:
def findtopspots(state):
    res = []
    global famousspots
    if state not in famousspots.keys():
        return ["Not Available"]
    places = famousspots[state]
    for place in places:
        if place[0] !="nan":
            res.append(place[0])
    return res[1:10]        

In [42]:
def create_window():
    window = Toplevel(root)
    window.title("Popular places in this state!")
    window.geometry("500x500")
    window.configure(background='pink')
    state=e1.get()
    op_list=findtopspots(state)
    for i in range(len(op_list)):
        l = Label(window, text=(str(i+1)+"."+op_list[i]))
        l.pack()

In [43]:
def similarityList():
    window = Toplevel(root)
    window.title("similar places based on popularity")
    window.geometry("500x500")
    window.configure(background='red')
    place=e2.get()
    op2_list=findsimilarspots(place)
    State=['newyork', 'alaska', 'california']
    for i in range(len(op2_list)):
        l = Label(window, text=(str(i+1)+"."+op2_list[i]))
        l.pack()

In [44]:
root = Tk()
root.title("Planning your next trip?")
root.geometry("700x700")
root.configure(background='yellow')
path = "US_Travel.jpg"
img = ImageTk.PhotoImage(Image.open(path))
label = Label(image=img)
label.image = img
label.pack(expand = "yes")

In [45]:
L1 = Label(root, text="Need Statewise Suggestions? Enter State")
L1.pack()
e1 = Entry(root, bd =5)
e1.pack()
b1 = Button(root, text="Enter", command=create_window)
b1.pack()

In [46]:
L2 = Label(root, text="Enter the name of your favorite tourist spot you have visited and loved!")
L2.pack()
e2 = Entry(root, bd =5)
e2.pack()
b2 = Button(root, text="check out similar places", command=similarityList)
b2.pack()

In [47]:
root.mainloop()